In [20]:
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)


install_and_import('einops')

In [21]:
import pandas as pd
input_file = '/kaggle/input/pipeline-dataset/chr14_training_dataset.csv'
# Load the dataset
df = pd.read_csv(input_file)

print(df)

                                                sequence  label
0      TAAAGAGACAAGCACTATATGCCTGTAGTACCTGCACCCCAGAGGG...      0
1      TGGTGGTCCGGGGCCGCCGTTAGTCTGCCGCTTCCTCACCCCAAGC...      0
2      GCTTGCAGTTATGAATGTCATGGAAGTCACAAACTATTTGAGCTAT...      1
3      ATCTTGGTGCCATTCCCCCTGAGAGTGGGGATTTTCAGGAGATGGT...      0
4      ACCTTTTGAAACCATTGTTGTCACAAAAAGATAAAGGTTATATTTT...      0
...                                                  ...    ...
72673  atacaggagcacccagattcataaagcaagtccttagagacctaca...      0
72674  AGAGTCAAGTCCCTGGGAACTTCAGGAAGCAGCccaccatgtttca...      1
72675  GAAAAGATGCATGAAGACACAGAAAAGAAACTAAAGGAAACAGCTG...      0
72676  ttatgggctggaacacttttatcttggcagtcccataatctagGAT...      0
72677  CTGGATCCAATGTTAGTCTTTTCACTTTCAGGAGATAGAAAAGATG...      1

[72678 rows x 2 columns]


In [22]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

dataset = Dataset.from_csv(input_file)
print(dataset)

train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['sequence', 'label'],
    num_rows: 72678
})


In [23]:
import pandas as pd

train_data = pd.DataFrame.from_dict(train_data)
val_data = pd.DataFrame.from_dict(val_data)

In [24]:
print(train_data)
print(val_data)

                                                sequence  label
0      CCAAATGGGATGTGTTTTTCTTATATACAAGTTGTTCACTTTGAAA...      0
1      AAAGACAGCCCCTCCATGTGAGGAAGCCCCCAACTGGGGAAAACCC...      0
2      CGTGTGCCATGTGGGGAAGGCGGAGGACCGGGAGCGGCTGGTGGCC...      1
3      gggcatgactgTAGTTGTTTCCCTCTGTAGTCGTAGTCGAGATCGG...      1
4      TAATGCCAGCTCAAAAACAGAGCTGACCATTTCTCTTTCATATGTT...      1
...                                                  ...    ...
58137  TTAAGGGTGATCTGCCAGATTGCGAAGCTGACCAACTCCTGCAGAT...      0
58138  AGGGAGGCAAATGCCTAACCTATGCTAAAGACGCATTATGCTAGGA...      0
58139  AGAACCTCCACTGACCTTCAGGAAGCAGGAGTGCTAGAAAGTACAG...      0
58140  GTTTCCTGTATTCCTGAGATCATTGTTTATTTTTCTTATTCTCTCT...      1
58141  TCCTTGACAAAACGAAAAACAATTGGGAAGATCGAGAAAAGTTTGA...      0

[58142 rows x 2 columns]
                                                sequence  label
0      TGCTCTGGCCCCTGGACTCACCGTGTGTTCTCCCTTCTTCCACCCC...      1
1      TTATAAAAATACTTAACATTCATTGTTTTTTTAAAAAAGACATTAT...      0
2      TGTCAGT

In [25]:
from transformers.models.bert.configuration_bert import BertConfig
from transformers import AutoModel, AutoTokenizer
import torch

tokenizer = tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

def tokenize_data(df, tokenizer):
    return tokenizer(df['sequence'].tolist(), padding=True, truncation=True, return_tensors='pt')

train_encodings = tokenize_data(train_data, tokenizer)
val_encodings = tokenize_data(val_data, tokenizer)

train_labels = torch.tensor(train_data['label'].values)
val_labels = torch.tensor(val_data['label'].values)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [26]:
print(len(train_labels))
print(len(val_labels))

58142
14536


In [27]:
from torch.utils.data import Dataset

class DnaDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DnaDataset(train_encodings, train_labels)
val_dataset = DnaDataset(val_encodings, val_labels)

print(train_dataset)
print(val_dataset)

In [28]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
wandb.login(key="WANDB_API_KEY")
wandb.init(project="dna-classification")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [29]:
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    report_to="wandb",
)


/root/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:

from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support

# Define compute_metrics function
def compute_metrics(pred):
    logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = logits.argmax(-1)
    labels = pred.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision, recall, _, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [32]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
50,0.670300
100,0.618400
150,0.618600
200,0.607700
250,0.597800
300,0.595400
350,0.627200
400,0.569100
450,0.509300
500,0.443900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=5451, training_loss=0.30919973940132867, metrics={'train_runtime': 1705.7094, 'train_samples_per_second': 102.26, 'train_steps_per_second': 3.196, 'total_flos': 1.2161114088910416e+16, 'train_loss': 0.30919973940132867, 'epoch': 3.0})

In [33]:
# Evaluate on test set
results = trainer.evaluate(eval_dataset=val_dataset)
print(results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.22098875045776367, 'eval_accuracy': 0.9287286736378646, 'eval_f1': 0.9285576496130549, 'eval_precision': 0.9284660023521114, 'eval_recall': 0.9287286736378646, 'eval_runtime': 38.705, 'eval_samples_per_second': 375.558, 'eval_steps_per_second': 2.945, 'epoch': 3.0}


In [34]:
# Finish the wandb run
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▁▂▂▂▃█▃▂▂▁▄▂▂▁▁▄▃▁▃▄▁▂▅▁▁▃▁▁▁▁▂▁▁▁▂▅▂▁▃▁
